In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
from pathlib import Path

In [41]:
DATA_DIR = Path("../data/")
SHARED_DIR = Path("/home/jovyan/workbench-shared-folder/bioblp/data")
Y_08_BENCHMARK_DIR = SHARED_DIR.joinpath("benchmarks/yamanashi/raw")
Y_08_BENCHMARK_DIR_INTERIM = SHARED_DIR.joinpath("benchmarks/yamanashi/interim")

dpi_enzyme_path = Y_08_BENCHMARK_DIR.joinpath("bind_orfhsa_drug_e.txt")
dpi_gpcr_path = Y_08_BENCHMARK_DIR.joinpath("bind_orfhsa_drug_gpcr.txt")
dpi_ic_path = Y_08_BENCHMARK_DIR.joinpath("bind_orfhsa_drug_ic.txt")
dpi_nr_path = Y_08_BENCHMARK_DIR.joinpath("bind_orfhsa_drug_nr.txt")


## Prepare ID maps

1. load Yamanishi08 dataset (clean, aggregate)
2. prepare id maps
    - KEGG Id to Uniprot (for proteins)
        - Use UniprotID MApper API
        - manually augment missing proteins
    - KEGG Id to Drugbank (for drugs)
        - :death, Kegg has become a commercial entity since then, so id maps are heavily locked down
        - scavenge for 2-hop projections kegg->pubmed->drugbank etc
        - retrieve drugabank data directly from website to find DB-Kegg maps
3. Map original Yamanishi08 dataset to new DB and Uniprot ids. 
4. Analyse missing data
5. access Mohammed 2020 datasets [PENDING Request]

### Load Yamanish 08 dataset

In [42]:
dpi_e_df = pd.read_csv(dpi_enzyme_path, sep="\t", names=["protein", "drug"])
dpi_ic_df = pd.read_csv(dpi_ic_path, sep="\t", names=["protein", "drug"])
dpi_gpcr_df = pd.read_csv(dpi_gpcr_path, sep="\t", names=["protein", "drug"])
dpi_nr_df = pd.read_csv(dpi_nr_path, sep="\t", names=["protein", "drug"])
dpi_e_df.head(2)


,protein,drug
0,hsa:10,D00002
1,hsa:10,D00448


In [43]:
dpi_yamanashi_all = pd.concat([dpi_e_df, dpi_ic_df, dpi_nr_df, dpi_gpcr_df], axis=0)
dpi_yamanashi_all.to_csv(Y_08_BENCHMARK_DIR_INTERIM.joinpath("dpi_yaminishi_08.tsv"), sep="\t", index=None, header=True)

In [6]:
# port drug kegg ids to a txt file for obtaining drugbank identifiers
dpi_yamanashi_all_drug = dpi_yamanashi_all[["drug"]]
dpi_yamanashi_all_drug = dpi_yamanashi_all_drug.drop_duplicates(["drug"])
print(f'# Unique drugs in Y_08 dataset: {len(dpi_yamanashi_all_drug)}')
#dpi_yamanashi_all_drug.to_csv(Y_08_BENCHMARK_DIR_INTERIM.joinpath("drug_kegg.txt"), sep="\t", index=None, header=False)


# Unique drugs in Y_08 dataset: 791


In [7]:
# port protein kegg ids to a txt file for obtaining uniprot ids using Uniprot ID mapping API (https://www.uniprot.org/id-mapping/uniprotkb/e5a1fb82351aeb0531bf2bf972c1362381eb26fc/overview?dir=ascend&facets=reviewed%3Atrue&sort=accession)
dpi_yamanashi_all_prot = dpi_yamanashi_all[["protein"]]
dpi_yamanashi_all_prot = dpi_yamanashi_all_prot.drop_duplicates(["protein"])
print(f'# Unique drugs in Y_08 dataset: {len(dpi_yamanashi_all_prot)}')
#dpi_yamanashi_all_prot.to_csv(Y_08_BENCHMARK_DIR_INTERIM.joinpath("protein_kegg.txt"), sep="\t", index=None, header=False)
dpi_yamanashi_all_prot.head(2)                            

# Unique drugs in Y_08 dataset: 989


,protein
0,hsa:10
2,hsa:100


#### Load Kegg:Uniprot id map

In [8]:
kegg2uniprot_path = Y_08_BENCHMARK_DIR_INTERIM.joinpath("uniprot-download_true_fields_accession_format_tsv_query__28reviewed_-2022.09.12-20.25.33.31.tsv")
#df = pd.DataFrame(kegg2uniprot_path)

In [9]:
kegg2uniprot_df = pd.read_csv(kegg2uniprot_path, sep="\t", names=["kegg", "uniprot"], header=0)
kegg2uniprot_df.head()

,kegg,uniprot
0,hsa:285242,A5X5Y0
1,hsa:7084,O00142
2,hsa:3775,O00180
3,hsa:10747,O00187
4,hsa:2918,O00222


In [10]:
kegg2uniprot_dict = dict(zip(kegg2uniprot_df.kegg, kegg2uniprot_df.uniprot))


#### 5 missing ids:
* [hsa:1384](https://www.genome.jp/dbget-bin/www_bget?hsa:1384+H00833)
    * P43155 mapped via [HGNC 2342](https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:2342)

* hsa:94009
    * [Q9NQF3](https://www.uniprot.org/uniprotkb/Q9NQF3/entry) mapped via [HAPPI](http://discovery.informatics.uab.edu/HAPPI/protein-description.php?protein=SERHL_HUMAN)

* [hsa:5447](https://www.genome.jp/dbget-bin/www_bget?hsa+5447)
    * [P16435](https://www.uniprot.org/uniprotkb/P16435/entry) mapped via [HGNC 9208](https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:9208)
    
* [hsa:6096](https://www.genome.jp/dbget-bin/www_bget?hsa:6096)
    * [Q92753](https://www.uniprot.org/uniprotkb/Q92753/entry) mapped via [HGNC 10259](https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:10259)
    
* hsa:390956
    *  Possibly [P50406](https://www.uniprot.org/uniprotkb/P50406/entry) mapped via unclear link [R-HSA-390956](https://reactome.org/content/detail/R-HSA-390956)

In [11]:
manually_matched_prots = {"hsa:1384": "P43155",
                         "hsa:94009": "Q9NQF3",
                         "hsa:5447": "P16435",
                         "hsa:6096": "Q92753",
                         }
kegg2uniprot_dict.update(manually_matched_prots)

In [12]:
len(kegg2uniprot_dict)

987

#### Id maps to cross reference across databases

In [14]:
db_links_path = Y_08_BENCHMARK_DIR_INTERIM.joinpath("drug_links.csv")

In [15]:
db_mappings_df = pd.read_csv(db_links_path, sep=",")
num_db_drugs = len(db_mappings_df)
print(f'# DB rows: {num_db_drugs}')
db_mappings_df.head(2)


# DB rows: 14315


,DrugBank ID,Name,CAS Number,Drug Type,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,ChEBI ID,PharmGKB ID,...,GenBank ID,DPD ID,RxList Link,Pdrhealth Link,Wikipedia ID,Drugs.com Link,NDC ID,ChemSpider ID,BindingDB ID,TTD ID
0,DB00001,Lepirudin,138068-37-8,BiotechDrug,NaN,D06880,NaN,46507011.0,NaN,PA450195,...,NaN,11916,http://www.rxlist.com/cgi/generic/lepirudin.htm,NaN,Lepirudin,http://www.drugs.com/cdi/lepirudin.html,NaN,NaN,NaN,DAP000541
1,DB00002,Cetuximab,205923-56-4,BiotechDrug,NaN,D03455,NaN,46507042.0,NaN,PA10040,...,J00228,13175,http://www.rxlist.com/cgi/generic3/erbitux.htm,NaN,Cetuximab,http://www.drugs.com/cdi/cetuximab.html,NaN,NaN,NaN,DNC000788


In [16]:
# which columns can we reliably use to map db ids?
prop_kegg_na = sum(db_mappings_df['KEGG Drug ID'].isna())/num_db_drugs*100
print('% DB drugs without KEGG identifiers {:.2f}'.format(prop_kegg_na))

% DB drugs without KEGG identifiers 84.90


In [17]:
kegg2db_dict = dict(zip(db_mappings_df['KEGG Drug ID'], db_mappings_df['DrugBank ID']))
len(kegg2db_dict)

2163

## DPI benchmark

In [18]:
import numpy as np
from bioblp.data import COL_EDGE, COL_SOURCE, COL_TARGET
SHARED_DIR = Path("/home/jovyan/workbench-shared-folder/bioblp")

/home/jovyan/.conda/envs/bioblp-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
db2kegg_dict = {v:k for k,v in kegg2db_dict.items()}
uniprot2kegg_dict = {v:k for k,v in kegg2uniprot_dict.items()}

In [20]:
dpi_yamanashi_all["src_db"] = dpi_yamanashi_all["drug"].apply(lambda x: kegg2db_dict.get(x, np.nan))
dpi_yamanashi_all["tgt_uniprot"] = dpi_yamanashi_all["protein"].apply(lambda x: kegg2uniprot_dict.get(x, np.nan))

In [21]:
dpi_yamanashi_all.head(2)

,protein,drug,src_db,tgt_uniprot
0,hsa:10,D00002,NaN,P11245
1,hsa:10,D00448,DB00795,P11245


In [30]:
num_unmapped_drugs = dpi_yamanashi_all[dpi_yamanashi_all["src_db"].isna()]["drug"].nunique()
num_unmapped_prots = dpi_yamanashi_all[dpi_yamanashi_all["tgt_uniprot"].isna()]["protein"].nunique()

In [34]:
print(f'From Yamanashi_08 DPI dataset,\n#Drugs unmapped to Drugbank: {num_unmapped_drugs}, which is {num_unmapped_drugs/dpi_yamanashi_all["drug"].nunique()*100}%')
print(f'#Prots unmapped to Uniprot: {num_unmapped_prots}, which is {num_unmapped_prots/dpi_yamanashi_all["protein"].nunique()*100}%')

From Yamanashi_08 DPI dataset,
#Drugs unmapped to Drugbank: 303, which is 38.305941845764856%
#Prots unmapped to Uniprot: 2, which is 0.20222446916076847%


In [36]:
yamanishi_post = dpi_yamanashi_all.dropna(subset=["tgt_uniprot", "src_db"])

In [37]:
print(f'# Yamanashi_08 DPI pairs lost due to incomplete mapping from KEGG: {len(dpi_yamanashi_all)-len(yamanishi_post)}')

# Yamanashi_08 DPI pairs lost due to incomplete mapping from KEGG: 1414
